In [ ]:
# loading libraries

import pandas as pd
import numpy as np

In [ ]:
# loding train and test data

train_df = pd.read_csv('../input/commonlitreadabilityprize/train.csv')
test_df = pd.read_csv('../input/commonlitreadabilityprize/test.csv')

In [ ]:
# dropping some columns

train_df=train_df[['id','excerpt','target','standard_error']]

In [ ]:
!pip3 install ../input/sentence-transformers/sentence-transformers-master

In [ ]:
# loading model using sentence transformers

import sentence_transformers
from sentence_transformers import SentenceTransformer, models

# setting model path for fine-tuned roberta weights

model_path = '../input/finetuned-model1/checkpoint-568'
word_embedding_model = models.Transformer(model_path, max_seq_length=275)
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())
model = SentenceTransformer(modules=[word_embedding_model, pooling_model])


In [ ]:
# encoding train and test strings

X_train = model.encode(train_df.excerpt, device='cuda')
X_test = model.encode(test_df.excerpt, device='cuda')

In [ ]:
!pip3 install ../input/frozendict/frozendict-2.0.2-py3-none-any.whl
!pip3 install ../input/neuraltangent/neural_tangents-0.3.6-py2.py3-none-any.whl

In [ ]:
from sklearn.model_selection import StratifiedKFold
from datetime import datetime
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import BayesianRidge

from jax import random
from neural_tangents import stax
import neural_tangents as nt

train_df['Character Count'] = train_df['excerpt'].apply(lambda x: len(str(x)))

preds = []
train_scores = []

df_oof=train_df.copy()
df_oof['oof'] = 0

skf = StratifiedKFold(10, shuffle=True, random_state=42)

splits = list(skf.split(X=X_train, y=train_df['Character Count']))

# predicting out of fold scores for each fold and doing predictions for each training set

for i, (train_idx, val_idx) in enumerate(splits):
    print(f'\n------------- Training Fold {i + 1} / {10}')
    print("Current Time =", datetime.now().strftime("%H:%M:%S"))

    #clf = BayesianRidge(n_iter=300, verbose=True)
    #clf.fit(X_train[train_idx],train_df.target[train_idx])
    #train_score=mean_squared_error(train_df.target[train_idx], clf.predict(X_train[train_idx]), squared=False)
    #train_scores.append(train_score)
    #print(f"Fold {i} train RMSE: {train_score}")
    
    ResBlock = stax.serial(
                stax.FanOut(2),
                stax.parallel(
                    stax.serial(
                        stax.Erf(),
                        stax.Dense(1, W_std=1.25, b_std=0.0),
                        stax.Erf(),
                        stax.Dense(1, W_std=1.25, b_std=0.0),
                        stax.Erf(),
                        stax.Dense(1, W_std=1.25, b_std=0.0),
                    ),
                    stax.Identity(),
                ),
                stax.FanInSum()
            )

    init_fn, apply_fn, kernel_fn = stax.serial(
        stax.Dense(1, W_std=1.0, b_std=0),
        ResBlock, ResBlock, stax.Erf(),
        stax.Dense(1, W_std=2.5, b_std=0.1)
    )
    
    key = random.PRNGKey(10)
    _, params = init_fn(key, input_shape=X_train[train_idx].shape)
    predict_fn = nt.predict.gradient_descent_mse_ensemble(kernel_fn,
                                                          X_train[train_idx],
                                                          train_df.target[train_idx].values[:,np.newaxis],
                                                          diag_reg=1e-1,
                                                          lr=1)
    
    
    train_score=mean_squared_error(train_df.target[train_idx], predict_fn(x_test=X_train[train_idx], get='ntk', t=None), squared=False)
    train_scores.append(train_score)
    print(f"Fold {i} train RMSE: {train_score}")
    
    preds.append(predict_fn(x_test=X_test, get='ntk', t=None))
    #x=clf.predict(predict_fn(x_test=X_train[val_idx], get='ntk', t=None))
    #df_oof['oof'].iloc[val_idx]+= x

print(f'Training score: {np.mean(train_scores)}, Training STD: {np.std(train_scores)}')
#print(f'OOF score across folds: {mean_squared_error(df_oof.target, df_oof.oof, squared=False)}')

In [ ]:
# getting mean prediction across 5 folds
y_pred = np.mean(preds,0)
y_pred.shape

In [ ]:
# creating submission csv

sub = test_df[["id"]].copy()
sub["target"] = y_pred
sub.to_csv('submission.csv', index = False)

In [ ]:
# checking submission file

sub.head()